In [1]:
import pandas as pd
import numpy as np
import requests
import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats
from itertools import combinations
from scipy.stats import shapiro

In [3]:
venta = pd.read_csv("inmuebles_venta.csv")

In [4]:
df = venta.copy()


In [5]:
df.info() 

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 9683 entries, 0 to 9682
Data columns (total 12 columns):
 #   Column                 Non-Null Count  Dtype 
---  ------                 --------------  ----- 
 0   id                     9683 non-null   object
 1   Descripción            9683 non-null   object
 2   Localización           9683 non-null   object
 3   Enlace                 9683 non-null   object
 4   Precio                 9350 non-null   object
 5   Superficie Construida  9350 non-null   object
 6   Última Actualización   9350 non-null   object
 7   Consumo Energético     2078 non-null   object
 8   Emisiones CO2          2078 non-null   object
 9   Características        9350 non-null   object
 10  Tipo de operación      9683 non-null   object
 11  timestamp_scrapeo      9683 non-null   object
dtypes: object(12)
memory usage: 907.9+ KB


In [6]:
# Función para extraer las características de la columna y convertirlas en columnas separadas
def extraer_caracteristicas(fila):
    # Elimina los caracteres adicionales para convertir la cadena en una lista de tuplas
    fila = fila.strip("[]")  # Elimina los corchetes externos
    elementos = fila.split("), (")  # Divide la cadena en tuplas individuales
    
    datos_extraidos = {}
    for elemento in elementos:
        # Divide cada elemento en clave y valor
        clave, valor = elemento.split(", ", 1)
        clave = clave.replace("(", "").replace("'", "").strip()  # Limpia clave
        valor = valor.replace(")", "").replace("'", "").strip()  # Limpia valor
        datos_extraidos[clave.strip(':')] = valor  # Remueve ':' del final de la clave si está presente
    return datos_extraidos

# Aplicar la función a la columna 'Características' y expandir en columnas separadas
caracteristicas_df = df['Características'].dropna().apply(extraer_caracteristicas)
caracteristicas_expandidas = pd.json_normalize(caracteristicas_df)

# Combinar las columnas expandidas con el DataFrame original (elimina la columna 'Características' original)
df_limpio = pd.concat([df.drop(columns=['Características']), caracteristicas_expandidas], axis=1)

# Guardar el DataFrame limpio en un nuevo archivo CSV si deseas conservarlo
df_limpio.to_csv('venta_inmuebles_limpio.csv', index=False)

# Visualización del resultado
print(df_limpio.head())


                                     id  \
0  bfcba5e4-5958-4445-9de7-0ab4df732b8c   
1  a0f3e8d5-ca6f-45c5-a5d4-2d3d6b7bbd22   
2  2efd9d2c-cdc9-45a6-91eb-6173f0e94ac7   
3  15c5ec6e-b28e-42d1-8fb9-497fa3158469   
4  a0676ed8-fa97-44ac-b242-aa947e8a2016   

                                         Descripción  \
0  Chalet en venta en Boadilla del Monte - Parque...   
1                    Casa pareada en venta en Cobeña   
2    Piso en venta en Calle Joaquim Blume, Número 14   
3  Piso en venta en Avenida de Concha Espina, cer...   
4                 Piso en venta en Calle del Duratón   

                                        Localización  \
0            Parque de Boadilla (Boadilla del Monte)   
1                                             Cobeña   
2                  Zona Suroeste (Torrejón de Ardoz)   
3  Hispanoamérica (Distrito Chamartín. Madrid Cap...   
4           Lucero (Distrito Latina. Madrid Capital)   

                                              Enlace     Precio  \


In [7]:
df_limpio

,id,Descripción,Localización,Enlace,Precio,Superficie Construida,Última Actualización,Consumo Energético,Emisiones CO2,Tipo de operación,...,Baños,Antigüedad,Conservación,Referencia,Tipo de casa,Superficie útil,Planta,Gastos de comunidad,Interior,Exterior
0,bfcba5e4-5958-4445-9de7-0ab4df732b8c,Chalet en venta en Boadilla del Monte - Parque...,Parque de Boadilla (Boadilla del Monte),https://www.pisos.com/comprar/chalet-parque_de...,1.795.000,415,Anuncio actualizado el 05/11/2024,Consumo:104 kWh/m² año,Emisiones:53 Kg CO₂/m² año,compra,...,5,Entre 10 y 20 años,En buen estado,IF5271-181,NaN,NaN,NaN,NaN,NaN,NaN
1,a0f3e8d5-ca6f-45c5-a5d4-2d3d6b7bbd22,Casa pareada en venta en Cobeña,Cobeña,https://www.pisos.com/comprar/casa_pareada-cob...,670.000,360,Anuncio actualizado el 05/11/2024,Consumo:104 kWh/m² año,Emisiones:53 Kg CO₂/m² año,compra,...,3,Entre 10 y 20 años,En buen estado,IF5271-206,Pareada,NaN,NaN,NaN,NaN,NaN
2,2efd9d2c-cdc9-45a6-91eb-6173f0e94ac7,"Piso en venta en Calle Joaquim Blume, Número 14",Zona Suroeste (Torrejón de Ardoz),https://www.pisos.com/comprar/piso-zona_suroes...,254.900,103,Anuncio actualizado el 02/11/2024,NaN,NaN,compra,...,2,Entre 20 y 30 años,En buen estado,4525678-000058,NaN,92 m²,1ª,Entre 60 y 80 €,NaN,NaN
3,15c5ec6e-b28e-42d1-8fb9-497fa3158469,"Piso en venta en Avenida de Concha Espina, cer...",Hispanoamérica (Distrito Chamartín. Madrid Cap...,https://www.pisos.com/comprar/piso-chamartin_h...,950.000,141,Anuncio actualizado el 05/11/2024,Consumo:104 kWh/m² año,Emisiones:56 Kg CO₂/m² año,compra,...,2,Más de 50 años,En buen estado,IF5271-51,NaN,NaN,1ª,NaN,NaN,NaN
4,a0676ed8-fa97-44ac-b242-aa947e8a2016,Piso en venta en Calle del Duratón,Lucero (Distrito Latina. Madrid Capital),https://www.pisos.com/comprar/piso-lucero28011...,210.000,65,Anuncio actualizado el 17/10/2024,NaN,NaN,compra,...,1,NaN,NaN,STA16-RP1212024128799,NaN,59 m²,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
9678,ed977a06-35d6-4d05-bed2-b3a7afc07290,"Casas y pisos en Aravaca, Distrito Moncloa-Ara...",Sé el primero en enterarte,https://www.pisos.com/comprar/piso-moncloa_ara...,NaN,NaN,NaN,NaN,NaN,compra,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9679,9796437d-4616-43fd-a0c5-590a338d2678,"Casas y pisos en Aravaca, Distrito Moncloa-Ara...",Sé el primero en enterarte,https://www.pisos.com/comprar/piso-moncloa_ara...,NaN,NaN,NaN,NaN,NaN,compra,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9680,f5c76a54-c3bf-445d-ba23-4b516989214d,"Casas y pisos en Aravaca, Distrito Moncloa-Ara...",Sé el primero en enterarte,https://www.pisos.com/comprar/piso-moncloa_ara...,NaN,NaN,NaN,NaN,NaN,compra,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
9681,99ddc50b-86a9-4e62-8119-f8aa293ca866,"Casas y pisos en Aravaca, Distrito Moncloa-Ara...",Sé el primero en enterarte,https://www.pisos.com/comprar/piso-moncloa_ara...,NaN,NaN,NaN,NaN,NaN,compra,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [8]:
# Lista de tipos de casa que queremos identificar
tipos_casa = ["Piso", "Chalet", "Adosado", "Casa", "Estudio", "Dúplex"]

# Función para extraer el tipo de casa de la descripción
def extraer_tipo_casa(descripcion):
    for tipo in tipos_casa:
        if tipo.lower() in descripcion.lower():
            return tipo
    return "Otro"  # Valor por defecto si no coincide con ninguno de los tipos en la lista

# Crear una nueva columna en el DataFrame con el tipo de casa
df_limpio['Tipo de Casa'] = df_limpio['Descripción'].apply(extraer_tipo_casa)

# Visualización del resultado
print(df_limpio[['Descripción', 'Tipo de Casa']].head())


                                         Descripción Tipo de Casa
0  Chalet en venta en Boadilla del Monte - Parque...       Chalet
1                    Casa pareada en venta en Cobeña         Casa
2    Piso en venta en Calle Joaquim Blume, Número 14         Piso
3  Piso en venta en Avenida de Concha Espina, cer...         Piso
4                 Piso en venta en Calle del Duratón         Piso


In [9]:
# Extraer la fecha de la columna 'Última Actualización' y convertirla a tipo datetime
df_limpio['Última Actualización'] = df_limpio['Última Actualización'].str.extract(r'(\d{2}/\d{2}/\d{4})')
df_limpio['Última Actualización'] = pd.to_datetime(df_limpio['Última Actualización'], format='%d/%m/%Y')

# Verificar el resultado
print(df_limpio['Última Actualización'].head())


0   2024-11-05
1   2024-11-05
2   2024-11-02
3   2024-11-05
4   2024-10-17
Name: Última Actualización, dtype: datetime64[ns]


In [12]:
# Eliminar filas donde 'Precio' es NaN
df_limpio = df_limpio.dropna(subset=['Precio'])


In [13]:
# Filtrar filas en 'Precio' que contienen caracteres no numéricos
valores_no_numericos = df_limpio[~df_limpio['Precio'].str.replace('.', '', regex=False).str.replace(',', '', regex=False).str.isnumeric()]

# Mostrar los resultados
print(valores_no_numericos[['Precio']])


     Precio
601       A
930       A
1183      A
1199      A
1321      A
...     ...
9324      A
9325      A
9329      A
9330      A
9331      A

[61 rows x 1 columns]


In [16]:
df_limpio['Precio'] = pd.to_numeric(
    df_limpio['Precio'].replace({r'[^\d,.-]': ''}, regex=True).str.replace(',', '.'), errors='coerce'
)

# Eliminar filas donde 'Precio' es NaN (valores no convertibles)
df_limpio = df_limpio.dropna(subset=['Precio'])

print(df_limpio['Precio'].head())

1    670.00
2    254.90
3    950.00
4    210.00
5    498.95
Name: Precio, dtype: float64


In [17]:
# Rellenar valores nulos en columnas categóricas
df_limpio['Antigüedad'].fillna('No especificado', inplace=True)
df_limpio['Conservación'].fillna('No especificado', inplace=True)
df_limpio['Planta'].fillna('No especificado', inplace=True)

/var/folders/jb/v4t6swb9003c33x4phtghtpc0000gn/T/ipykernel_56488/911444348.py:2: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are setting values always behaves as a copy.

For example, when doing 'df[col].method(value, inplace=True)', try using 'df.method({col: value}, inplace=True)' or df[col] = df[col].method(value) instead, to perform the operation inplace on the original object.


  df_limpio['Antigüedad'].fillna('No especificado', inplace=True)
/var/folders/jb/v4t6swb9003c33x4phtghtpc0000gn/T/ipykernel_56488/911444348.py:3: FutureWarning: A value is trying to be set on a copy of a DataFrame or Series through chained assignment using an inplace method.
The behavior will change in pandas 3.0. This inplace method will never work because the intermediate object on which we are settin

In [18]:
 #Convertir 'Superficie construida' y 'Superficie útil' a numérico
df_limpio['Superficie construida'] = df_limpio['Superficie construida'].str.replace(' m²', '').astype(float)
df_limpio['Superficie útil'] = df_limpio['Superficie útil'].str.replace(' m²', '').astype(float)

In [19]:
# Eliminar filas duplicadas
df_limpio.drop_duplicates(inplace=True)


In [20]:
# Usar .title() para capitalizar la primera letra de cada palabra en los nombres de columna
df_limpio.columns = df_limpio.columns.str.title()

# Verificar los nombres de las columnas
print(df_limpio.columns)


Index(['Id', 'Descripción', 'Localización', 'Enlace', 'Precio',
       'Superficie Construida', 'Última Actualización', 'Consumo Energético',
       'Emisiones Co2', 'Tipo De Operación', 'Timestamp_Scrapeo',
       'Superficie Construida', 'Superficie Solar', 'Habitaciones', 'Baños',
       'Antigüedad', 'Conservación', 'Referencia', 'Tipo De Casa',
       'Superficie Útil', 'Planta', 'Gastos De Comunidad', 'Interior',
       'Exterior', 'Tipo De Casa'],
      dtype='object')


In [21]:
codigos_postales_madrid = ({
    # Barrios de Madrid Capital
    'Sol': '28013', 'Acacias': '28045', 'Ibiza': '28009', 'Recoletos': '28001', 'El Viso': '28002',
    'Bellavista': '28020', 'Almagro': '28010', 'La Paz': '28029', 'Argüelles': '28008', 'Los Cármenes': '28047',
    'Opañel': '28019', 'Orcasitas': '28026', 'Palomeras Bajas': '28018', 'Pavones': '28030', 'Ventas': '28017',
    'Pinar del Rey': '28033', 'San Andrés': '28021', 'Casco Histórico de Vallecas': '28031', 'Ambroz': '28032',
    'Simancas': '28037', 'Timón': '28042', 'Malasaña': '28004', 'Chueca': '28004', 'Lavapiés': '28012',
    'Embajadores': '28012', 'Tetuán': '28039', 'Cuatro Caminos': '28020', 'Chamberí': '28015', 'Salamanca': '28006',
    'Arganzuela': '28045', 'Retiro': '28007', 'Ciudad Universitaria': '28040', 'Hortaleza': '28033', 
    'Moncloa': '28008', 'Moratalaz': '28030', 'Carabanchel': '28025', 'Villaverde': '28021', 'Usera': '28026', 
    'Barajas': '28042', 'Canillejas': '28022', 'Vallecas': '28038', 'Puente de Vallecas': '28053', 
    'Vicálvaro': '28032', 'Alameda de Osuna': '28042', 'La Moraleja': '28109', 'Piovera': '28043', 
    'Ciudad Lineal': '28027', 'Ensanche de Vallecas': '28051', 'Las Tablas': '28050', 'Sanchinarro': '28050', 
    'Aravaca': '28023', 'El Pardo': '28048', 'Montecarmero': '28049', 'Prosperidad': '28002', 'Palacio': '28013',
    'Hispanoamérica': '28036', 'Arcos': '28037', 'Castilla': '28046', 'Ciudad Jardín': '28002', 
    'Cortes-Huertas': '28014', 'Nueva España': '28016', 'Puerta del Ángel': '28011', 'Niño Jesús': '28009',
    'Rejas': '28022',"Latina":"28047","Rosas":"28037",
    
    # Ubicaciones específicas basadas en descripciones de propiedades
    'Plaza del Doctor Laguna': '28009', 'José Ortega y Gasset': '28006', 'Núñez de Balboa': '28006',
    'Parla Este': '28983', 'Calle de la Higuera': '28492', 'Calle Cerceda': '28413',
    'Avenida de Calatalifa': '28670', 'Puente de Vallecas': '28053', 'Plaza del Niño Jesús': '28009',
    'Calle de Max Aub': '28015', 'Almendrales': '28026', 'Calle de Cuart de Poblet': '28047', 
    'Hellín': '28037', 'Calle Hermano Crisanto': '28180', 'Fuentebella': '28980', 'Corpa': '28811',
    'Villayuventus-Renfe': '28980', 'Calle Viena': '28891', 'San Martín de La Vega': '28330', 
    'El Molar': '28710', 'Calle de Siro Muela': '28037', 'Guadarrama': '28440', 'Ajalvir': '28864', 
    'Pryconsa, Juan de Austria': '28805', 'Robledo de Chavela': '28294', 'Villanueva del Pardillo': '28229', 
    'Calle de Benito Pérez Galdós': '28991', 'Urb. Vallefresno': '28440', 'Campamento': '28024', 
    'Calle del Ánsar': '28047', 'Velilla de San Antonio': '28891', 'Amposta': '28037', 
    'Las Américas': '28983', 'Casco Antiguo Sur': '28350', 'Puerta de Madrid, El Juncal': '28805', 
    'Fuente El Saz de Jarama': '28140', 'El Vellón': '28754', 'Alonso Cano': '28010',
    "Piso en alquiler en Plaza del Doctor Laguna, 4": "28009",
    "Piso en alquiler en Calle de José Ortega y Gasset": "28006",
    "Piso en alquiler en Calle de Cuart de Poblet, Lucero": "28047",
    "Casa en alquiler en Calle Hermano Crisanto, Torrelaguna": "28180",
    "Piso en alquiler en Calle de Núñez de Balboa": "28006",
    "Piso en alquiler en Calle del Ánsar, Cármenes (Distrito Latina)": "28047",
    "Piso en alquiler en Parla Este": "28983",
    "Piso en alquiler en Calle de José Ortega y Gasset (Goya)": "28006",
    "Apartamento en alquiler en Lucero (Distrito Latina)": "28047",
    "Piso en alquiler en Calle de la Higuera": "28492",
    "Chalet en alquiler en Fuente El Saz de Jarama": "28140",
    "Piso en alquiler en Cármenes (Distrito Latina)": "28047",
    "Ático en alquiler en San Blas, Rosas (Distrito San Blas)": "28037",
    "Casa en alquiler en Calle de Siro Muela, Salvador (Distrito San Blas)": "28037",
    "Chalet en alquiler en Calle Cerceda": "28413",
    "Piso en alquiler en Lucero (Distrito Latina)": "28047",
    "Ático en alquiler en Calle de Alonso Cano": "28010",
    "Piso en alquiler en Avenida de Calatalifa": "28670",
    "Piso en alquiler en Puente de Vallecas": "28053",
    "Apartamento en alquiler en Almendrales": "28026",
    "Apartamento en alquiler en Alameda de Osuna": "28042",
    "Casa adosada en alquiler en Calle de Benito Pérez Galdós, Torrejón de la Calzada": "28991",
    "Piso en alquiler en Ibiza": "28009",
    "Piso en alquiler en Ciudad Lineal": "28027",
    "Piso en alquiler en Vallecas": "28038",
    "Ático en alquiler en Plaza del Niño Jesús, 6": "28009",
    "Piso en alquiler en Calle de Max Aub": "28015",
    
    # Municipios y áreas cercanas a Madrid
    'Boadilla del Monte': '28660', 'Pozuelo de Alarcón': '28223', 'Las Rozas de Madrid': '28231', 
    'Villanueva de la Cañada': '28691', 'Majadahonda': '28220', 'Alcobendas': '28100', 
    'San Sebastián de los Reyes': '28701', 'Getafe': '28901', 'Leganés': '28911', 'Fuenlabrada': '28940', 
    'Móstoles': '28931', 'Alcorcón': '28921', 'Rivas-Vaciamadrid': '28521', 'Coslada': '28821', 
    'San Fernando de Henares': '28830', 'Tres Cantos': '28760', 'Colmenar Viejo': '28770', 
    'Torrejón de Ardoz': '28850', 'Algete': '28110', 'Paracuellos de Jarama': '28860', 'Pinto': '28320', 
    'Valdemoro': '28341', 'Arganda del Rey': '28500', 'Galapagar': '28260', 'Cercedilla': '28470',
    'Villaviciosa de Odón': '28670', 'Bustarviejo': '28720', 'El Escorial': '28280', 'Navacerrada': '28491',
    'Collado Villalba': '28400', 'Moralzarzal': '28411', 'Nuevo Baztán': '28514', 'Meco': '28880',
    'Garena': '28806', 'Castillo': '28692', 'Centro': '28300', 'Las Matas': '28290', 'Torrelodones': '28250',
    'Espartales': '28805', 'Villalbilla': '28810', 'Los Robles': '28250', 'Sevilla la Nueva': '28609',
    'Brunete': '28690', 'Hoyo de Manzanares': '28240', 'Moraleja de Enmedio': '28950',
    'Alpedrete': '28430', 'Mejorada del Campo': '28840', 'Valdemorillo': '28210', 'Valdeavero': '28816',
    'Cobeña': '28863', 'Las Aves-Jardín de Oñate': '28300', 'Los Molinos': '28460', 'Nueva Alcalá': '28807',
    'Colmenarejo': '28270', 'Navalcarnero': '28600', 'El Berrueco': '28192', 'Rinconada': '28803',
    'Humanes de Madrid': '28970',
})

# Función para obtener el código postal según la localización
def obtener_codigo_postal(localizacion):
    for lugar in codigos_postales_madrid:
        if lugar in localizacion:
            return codigos_postales_madrid[lugar]
    return None

# Aplicar la función a la columna de localización y crear la columna 'CP'
df['CP'] = df['Localización'].apply(obtener_codigo_postal)

# Guardar el nuevo archivo CSV con los códigos postales
output_path = 'inmuebles_venta_con_cp.csv'
df.to_csv(output_path, index=False)

print(f'Archivo guardado en {output_path}')

Archivo guardado en inmuebles_venta_con_cp.csv
